In [ ]:
"""
Embed tokens in the dataset
"""
# path to raw dataset
raw_dataset_path = './stlawu-webpages.jsonl'
print(raw_dataset_path)

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('json', data_files=raw_dataset_path, split='train')
raw_dataset

raw_dataset['url'][:1000]


In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "intfloat/e5-mistral-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
# import torch

# device = torch.device("cuda")
# model.to(device)

In [ ]:
model.max_seq_length = 4096 * 2
print(model)
print(tokenizer)

In [ ]:

tokenized_text = tokenizer(raw_dataset[0]['html_doc'])
tokenized_text.input_ids[:10]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', truncation=True, return_tensors="pt", max_length=model.max_seq_length
    )
    print(encoded_input)
    # encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    return model_output


In [ ]:
get_embeddings(raw_dataset['html_doc'][0]).last_hidden_state.shape
get_embeddings('Hi, this is me!').last_hidden_state.shape

In [ ]:
from html_cleaner import clean_html

In [ ]:
print(raw_dataset['html_doc'][0])

In [ ]:
print(clean_html(raw_dataset['html_doc'][0]))

In [ ]:
cleaned_html = clean_html(raw_dataset['html_doc'][20])

embeddings = get_embeddings(cleaned_html)
print(embeddings)


In [ ]:
print(embeddings.last_hidden_state.shape)
print(embeddings.last_hidden_state[0, :-3, :])

In [ ]:
from html_cleaner import clean_dataset_path
from datasets import load_from_disk
clean_dataset = load_from_disk(clean_dataset_path)

In [ ]:
print(clean_dataset)

In [ ]:
# getting rid off unknown stuff
clean_dataset = clean_dataset.filter(lambda x: '<div>' in x['html_doc'])

In [ ]:
clean_dataset[4421]

In [25]:
# adding faiss index
from create_embeddings import embeddings_dataset_path
from datasets import load_from_disk
import numpy as np

embeddings_dataset = load_from_disk(embeddings_dataset_path)

In [26]:
# index 0 because embeddings are in batch
embeddings_dataset = embeddings_dataset.map(lambda x: {'embeddings' : np.array(x['embeddings'][0])})

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657/7657 [00:16<00:00, 471.44 examples/s]


In [21]:
print(embeddings_dataset['embeddings'][0])
# print(type(embeddings_dataset['embeddings'][0]))

[1.3046875, 4.36328125, 4.5703125, -3.875, -4.50390625, -4.76171875, -0.146240234375, 6.328125, 7.35546875, 5.14453125, 0.73583984375, 1.658203125, 1.6982421875, 3.296875, -2.712890625, -5.74609375, 8.40625, 2.880859375, 1.482421875, -7.4296875, -8.6328125, -4.375, 2.26953125, -2.4765625, 0.1263427734375, 0.501953125, -0.67724609375, -7.8671875, -0.85400390625, 2.85546875, 3.96875, 1.8662109375, 9.4375, 1.978515625, 0.73876953125, -4.59375, -0.81103515625, 2.099609375, -6.25, -0.002506256103515625, -4.22265625, 0.07489013671875, -0.1387939453125, 2.875, 1.23046875, 1.888671875, 6.8125, 6.1953125, -7.109375, -1.6923828125, 0.69677734375, -8.3671875, -1.7939453125, 32.625, -1.4150390625, -2.439453125, 7.609375, -5.046875, 3.587890625, -10.5546875, -2.716796875, 2.7578125, 1.20703125, -0.09063720703125, 4.3125, -1.73046875, -2.541015625, -3.97265625, -3.26953125, -4.96484375, -3.865234375, -1.8701171875, -6.4609375, 6.0703125, -1.716796875, -5.2109375, 0.953125, -0.1534423828125, -3.71875

In [22]:
embeddings_dataset.add_faiss_index(column='embeddings')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 39.91it/s]


Dataset({
    features: ['url', 'html_doc', 'embeddings'],
    num_rows: 7657
})

In [24]:
print(embeddings_dataset)

Dataset({
    features: ['url', 'html_doc', 'embeddings'],
    num_rows: 7657
})


In [27]:
# saving it 
embeddings_dataset.save_to_disk(embeddings_dataset_path + )

PermissionError: Tried to overwrite /home/nikolai/SearchEngine/stlawu-webpages-with-embeddings but a dataset can't overwrite itself.